In [ ]:
# With Anaconda
!conda install ipykernel requests
!pip install wget

# Without Anaconda
#! pip install ipykernel wget requests

In [9]:
import wget, os

# Select databases you want to use for training the model
DATASETS_SELECTED = [0, 1, 2]

# http://help.sentiment140.com/for-students - 10,000
# https://nlp.stanford.edu/sentiment/code.html
# https://ieee-dataport.org/open-access/stock-market-tweets-data#files - Hosted on own Nextcloud server
all_urls = ["http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip", "http://nlp.stanford.edu/~socherr/stanfordSentimentTreebank.zip", "https://nextcloud.lucashost.nl/s/bm8bqeKqdyFqiTe/download/tweets.zip"]
urls = [all_urls[i] for i in DATASETS_SELECTED]
path = "datasets/"

isExist = os.path.exists(path)
if not isExist:
  os.makedirs(path)

for url in urls:
    wget.download(url, out = path)

['http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip', 'http://nlp.stanford.edu/~socherr/stanfordSentimentTreebank.zip', 'https://nextcloud.lucashost.nl/s/bm8bqeKqdyFqiTe/download/tweets.zip']


In [6]:
import zipfile

for filename in os.listdir(path):
    if filename.endswith(".zip"):
        with zipfile.ZipFile(path + filename, 'r') as zip_ref:
            zip_ref.extractall(path)
        os.remove(path + filename)

In [1]:
from auth import CLIENT_ID, SECRET_KEY, REDDIT_USERNAME, REDDIT_PASSWORD

import requests

auth = requests.auth.HTTPBasicAuth(CLIENT_ID, SECRET_KEY)

data = {
    "grant_type": "password",
    "username": REDDIT_USERNAME,
    "password": REDDIT_PASSWORD
}

headers = {"User-Agent": "MyAPI/0.0.1"}

res = requests.post("https://www.reddit.com/api/v1/access_token", auth=auth, data=data, headers=headers)

TOKEN = res.json()["access_token"]

headers["Authorization"] = f'bearer {TOKEN}'

requests.get("https://oauth.reddit.com/api/v1/me", headers=headers).json()

{'is_employee': False,
 'seen_layout_switch': False,
 'has_visited_new_profile': False,
 'pref_no_profanity': True,
 'has_external_account': False,
 'pref_geopopular': '',
 'seen_redesign_modal': False,
 'pref_show_trending': True,
 'subreddit': {'default_set': True,
  'user_is_contributor': False,
  'banner_img': '',
  'restrict_posting': True,
  'user_is_banned': False,
  'free_form_reports': True,
  'community_icon': None,
  'show_media': True,
  'icon_color': '',
  'user_is_muted': False,
  'display_name': 'u_Rubinjo_L',
  'header_img': None,
  'title': '',
  'coins': 0,
  'previous_names': [],
  'over_18': False,
  'icon_size': [256, 256],
  'primary_color': '',
  'icon_img': 'https://styles.redditmedia.com/t5_5ga0v8/styles/profileIcon_snoo7e1baabd-7daf-4774-a95d-92829e511322-headshot.png?width=256&amp;height=256&amp;crop=256:256,smart&amp;s=5bf6cac6695af43722caff15194db1958f1fc158',
  'description': '',
  'submit_link_label': '',
  'header_size': None,
  'restrict_commenting': Fa

In [2]:
res = requests.get("https://oauth.reddit.com/r/python/hot", headers=headers)
res.json()

{'kind': 'Listing',
 'data': {'after': 't3_rattcx',
  'dist': 27,
  'modhash': None,
  'geo_filter': None,
  'children': [{'kind': 't3',
    'data': {'approved_at_utc': None,
     'subreddit': 'Python',
     'selftext': "It's (almost) December, which means it's time for [Advent of Code](https://adventofcode.com/)! \n\n[Advent of Code](https://adventofcode.com/2021/about) is a yearly advent calendar where there's a new programming puzzle each day. They vary in difficulty and don't require a computer science background to solve. You can solve the problems using any programming language you like, so while we love python here, you can take this as an opportunity to explore other languages. If you have trouble with one of the puzzles, don't worry, another day's puzzle might be easier for you!\n\nThis year we're joining our sister community, /r/LearnPython, for a shared leaderboard. The /r/Python and /r/LearnPython leaderboard code is: `1179846-33292986`\n\nIf you want to talk about your sol